# Floor Plan Generator

This notebook provides an interface to generate floor plans. You can input a text prompt, choose the number of images to generate, select the inference steps, and pick the scheduler.


### Istall required libraries

In [32]:
#!pip install torch diffusers ipywidgets


### Imports

In [63]:
import PIL
import imageio
import diffusers
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from ipywidgets import interactive, fixed
from IPython.display import Image, display, clear_output


import torch
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image.fid import FrechetInceptionDistance

from datasets import load_dataset
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from diffusers import (
    DiffusionPipeline,
    StableDiffusionPipeline,
    EulerDiscreteScheduler,
    PNDMScheduler,
    LMSDiscreteScheduler)
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run script troubleshoot

Jupyter command `jupyter-nbextension` not found.


### Image generation funtion

In [44]:
def generate_images(prompt, num_images, scheduler, steps):
    pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

    # Load the LoRA weights for the model
    pipeline.load_lora_weights("Checkpoint_L1/checkpoint-250", weight_name="pytorch_lora_weights.safetensors")

    # Set the scheduler (this part is a placeholder, please adapt to your needs)
    if scheduler == "DDIM":
        pipeline.scheduler = DDIMScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    # Add other schedulers as needed

    # Generate images based on the prompt
    images = []
    for _ in range(num_images):
        image = pipeline(prompt, num_inference_steps=steps, guidance_scale=7.5).images[0]
        images.append(image)
    return images



In [64]:
# Use cuda if GPU is available
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

print(f"Using device: {DEVICE}")

Using device: cuda


In [65]:
diffusion_model = DiffusionPipeline()


def load_model(model_name: str, device: str = DEVICE):
    """Function that loads a pretrained model from the HuggingFace model hub and moves it to the specified device

    Arguments:
        model_name (str): Name of the model to load
        device (str): Device to move the model to
    """

    # Define as global so that changes made within the function are also applied outside of it
    global diffusion_model

    diffusion_model = DiffusionPipeline.from_pretrained(
        pretrained_model_name_or_path=model_name
    )
    diffusion_model.to(device)

In [66]:
interactive(
    load_model,
    {"manual": True},
    model_name=[
        "anton-l/ddpm-butterflies-128",
        "anton-l/ddpm-ema-pokemon-64",
        "google/ddpm-ema-church-256",
        "google/ddpm-celebahq-256",
        "WiNE-iNEFF/Minecraft-Skin-Diffusion-V2",
    ],
    device=fixed(DEVICE),
)

interactive(children=(Dropdown(description='model_name', options=('anton-l/ddpm-butterflies-128', 'anton-l/ddp…

In [62]:
interactive(
    load_model,
    {"manual": True},
    model_name=[
        "anton-l/ddpm-butterflies-128",
        "anton-l/ddpm-ema-pokemon-64",
        "google/ddpm-ema-church-256",
        "google/ddpm-celebahq-256",
        "WiNE-iNEFF/Minecraft-Skin-Diffusion-V2",
    ],
    device=fixed(DEVICE),
)

interactive(children=(Dropdown(description='model_name', options=('anton-l/ddpm-butterflies-128', 'anton-l/ddp…

### Inputs

In [45]:
def display_images(prompt, num_images, steps, scheduler):
    images = generate_images(prompt, num_images, scheduler, steps)
    for idx, img in enumerate(images):
        display(img)
        img.save(f"generated_image_{idx+1}.png")

# Create interactive widgets
interactive_plot = interactive(display_images,
                               prompt=widgets.Textarea(value='', placeholder='Enter your prompt...', description='Prompt:'),
                               num_images=widgets.IntSlider(value=1, min=1, max=10, step=1, description='Num Images:'),
                               steps=widgets.IntSlider(value=50, min=1, max=100, step=1, description='Inference Steps:'),
                               scheduler=widgets.Dropdown(options=["DDIM", "PLMS", "Euler", "Euler a", "LMS", "Heun", "DPM2", "DPM2 a", "DPM++", "DPM++ a"], value='DDIM', description='Scheduler:'))

# Display the interactive widgets
display(interactive_plot)


interactive(children=(Textarea(value='', description='Prompt:', placeholder='Enter your prompt...'), IntSlider…

### Generate and display images